# Chapter 14 Generate Video Using Stable Diffusion

## 1. Technical requirements

In [5]:
%pip install -U diffusers==0.27.0
%pip install peft==0.6.2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.23.1
    Uninstalling diffusers-0.23.1:
      Successfully uninstalled diffusers-0.23.1
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


## 3. Practical applications of AnimateDiff

### 3.1. Load the AnimateDiff motion module

In [1]:
from diffusers import MotionAdapter
import torch

adapter = MotionAdapter.from_pretrained(
    "guoyww/animatediff-motion-adapter-v1-5-2"
)

/home/andrewzhu/storage_1t_1/az_git_folder/Using-Stable-Diffusion-with-Python/sd_book_prd_p310_venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


### 3.2. Load up a AnimateDiff pipeline from a Stable Diffusion v1.5 checkpoint model

In [2]:
from diffusers import AnimateDiffPipeline

# Use the model from Hugging Face repository directly
# model_path = "stablediffusionapi/majicmixrealistic-v6"

# use the model from local storage
model_path = "/home/andrewzhu/storage_1t_1/sd15_models/majicmixRealistic_v6"
pipe = AnimateDiffPipeline.from_pretrained(
    model_path
    , motion_adapter    = adapter
    , safety_checker    = None
)

Keyword arguments {'safety_checker': None} are not expected by AnimateDiffPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/andrewzhu/storage_1t_1/az_git_folder/Using-Stable-Diffusion-with-Python/sd_book_prd_p310_venv/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'dropout': 0.0, 'transformer_layers_per_block': 1, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'addition_time_embed_dim': None, 'num_class_embeds': None, 'upcast_attention': None, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_i

### 3.3. Prepare the scheduler

In [3]:
from diffusers import EulerAncestralDiscreteScheduler
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(
    model_path
    , subfolder         = "scheduler"
    , clip_sample       = False
    , timestep_spacing  = "linspace"
    , steps_offset      = 1
)
pipe.scheduler = scheduler
#pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_vae_tiling()

### 3.4. Generating a video clip

In [4]:
import torch 
from diffusers.utils import export_to_gif, export_to_video

prompt = """photorealistic, 1girl, dramatic lighting"""

neg_prompt = """worst quality, low quality, normal quality, lowres, bad anatomy, bad hands, monochrome, grayscale watermark, moles"""
pipe.to("cuda:0")

output = pipe(
    prompt                  = prompt
    , negative_prompt       = neg_prompt
    , height                = 256
    , width                 = 256
    , num_frames            = 16
    , num_inference_steps   = 30
    , guidance_scale        = 8.5
    , generator             = torch.Generator("cuda").manual_seed(7)
)
frames = output.frames[0]
torch.cuda.empty_cache()

export_to_gif(frames, "animation_origin_256_wo_lora.gif")
export_to_video(frames, "animation_origin_256_wo_lora.mp4")

  0%|          | 0/30 [00:00<?, ?it/s]

'animation_origin_256_wo_lora.mp4'

In [5]:
pipe.load_lora_weights("guoyww/animatediff-motion-lora-zoom-in", adapter_name="zoom-in")

pipe.load_lora_weights("guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out")

pipe.load_lora_weights("guoyww/animatediff-motion-lora-rolling-clockwise", adapter_name="rolling-clockwise")

pipe.load_lora_weights("guoyww/animatediff-motion-lora-tilt-up", adapter_name="tilt-up")

pipe.load_lora_weights("guoyww/animatediff-motion-lora-pan-left", adapter_name="pan-left")

diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

In [6]:
import torch 
from diffusers.utils import export_to_gif, export_to_video

pipe.to("cuda:0")
prompt = """
photorealistic, 1girl, shape, highres, dramatic lighting
"""

neg_prompt = """
worst quality, low quality, normal quality, lowres, bad anatomy, bad hands
, monochrome, grayscale watermark, watermark, url, site, names, moles, blur
"""

pipe.set_adapters(
    ["zoom-in", "zoom-out","rolling-clockwise","tilt-up","pan-left"]
    , adapter_weights = [
        1.0, 0.0, 0.0, 0.0, 0.0
    ]
)

output = pipe(
    prompt                  = prompt
    , negative_prompt       = neg_prompt
    , height                = 256
    , width                 = 256
    , num_frames            = 20
    , num_inference_steps   = 30
    , guidance_scale        = 8.5
    , generator             = torch.Generator("cuda").manual_seed(123)
)
frames = output.frames[0]
pipe.to("cpu")
torch.cuda.empty_cache()

export_to_gif(frames, "animation_origin_256_w_lora_zoom_in.gif")
export_to_video(frames, "animation_origin_256_w_lora_zoom_in.mp4")

  0%|          | 0/30 [00:00<?, ?it/s]

'animation_origin_256_w_lora_zoom_in.mp4'